# ENEA Technical Interview

Data insights from crash stats

In [2]:
import os
import functools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.width = 180
pd.options.display.max_colwidth = 120
pd.options.display.max_rows = 100

sns.reset_defaults()
sns.set_theme(context='notebook', style='ticks', font_scale=1, rc={
    'axes.spines.right': False,
    'axes.spines.top': False,
    'figure.figsize': (4,3),
})

print('numpy version', np.__version__)
print('pandas version', pd.__version__)

numpy version 1.21.6
pandas version 1.1.5


In [2]:
!unzip "2000 to 2005 ACCIDENT.zip" -d "2000 to 2005 ACCIDENT"
!unzip "ACCIDENT.zip" -d "2006 to 2020 ACCIDENT"

Archive:  2000 to 2005 ACCIDENT.zip
  inflating: 2000 to 2005 ACCIDENT/ACCIDENT.csv  
  inflating: 2000 to 2005 ACCIDENT/ACCIDENT_EVENT.csv  
  inflating: 2000 to 2005 ACCIDENT/ACCIDENT_LOCATION.csv  
  inflating: 2000 to 2005 ACCIDENT/NODE_ID_COMPLEX_INT_ID.csv  
  inflating: 2000 to 2005 ACCIDENT/ATMOSPHERIC_COND.csv  
  inflating: 2000 to 2005 ACCIDENT/NODE.csv  
  inflating: 2000 to 2005 ACCIDENT/PERSON.csv  
  inflating: 2000 to 2005 ACCIDENT/ROAD_SURFACE_COND.csv  
  inflating: 2000 to 2005 ACCIDENT/SUBDCA.csv  
  inflating: 2000 to 2005 ACCIDENT/VEHICLE.csv  
  inflating: 2000 to 2005 ACCIDENT/ACCIDENT_CHAINAGE.csv  
  inflating: 2000 to 2005 ACCIDENT/Statistic Checks.csv  
